# Enunciado


Un resorte helicoidal de compresión debe ejercer una fuerza de 8 lb cuando se comprime hasta una longitud de 1.75 in. 

Si la longitud es de 1.25 in, la fuerza debe ser de 12 lb. 
El resorte será instalado en una máquina que funciona con ciclos de lentitud, en un orificio de 0.75 in de diámetro, y se espera que tenga un total de 200,000 ciclos. 

La temperatura no será mayor que 200 °F.

Para esta aplicación, especifique un material, el diámetro de alambre, el diámetro medio, DE, DI, longitud libre, longitud comprimida, número de espiras y tipos de condiciones e los extremos adecuados. Comprobar el esfuerzo en la carga máxima de operación, y en la condición de longitud comprimida.

In [1123]:
import pandas as pd
from sympy import Eq, solve, Symbol
import numpy as np
from colorama import Fore

### Selección del material

In [1124]:
table_10_5 = pd.read_csv('tables/table_10_5.csv')
table_10_5.head()

,Material,Limite tension,limite torsion,Diámetro in,E M-psi,E GPa,G M-psi,G GPa
0,Alambre de piano A228,65-75,45-60,<0.032,29.5,203.4,12.00,82.7
1,Alambre de piano A228,65-75,45-60,0.033-0.063,29.0,200.0,11.85,81.7
2,Alambre de piano A228,65-75,45-60,0.064-0.125,28.5,196.5,11.75,81.0
3,Alambre de piano A228,65-75,45-60,>0.125,28.0,193.0,11.60,80.0
4,Resorte estirado duro A227,60-70,45-55,<0.032,28.8,198.6,11.70,80.7


In [1125]:
# Search Cromo-vanadio A231	
count = 0
material  = table_10_5[table_10_5['Material'] == 'Cromo-vanadio A231'].values[count][1:]
G = material[-2]
porcentaje_sut = int(material[1].split('-')[0])/100
material, porcentaje_sut, G

(array(['88-93', '65-75', nan, 29.5, 203.4, 11.2, 77.2], dtype=object),
 0.65,
 11.2)

### Establecer datos conocidos

In [1126]:
F_i = 8  # lb
L_i = 1.75 #in

F_o = 12 # lb
L_o = 1.25 # in

### Calculo de la constante del resorte

In [1127]:
k = (F_o - F_i)/(L_i - L_o)
print(f'k = {k :,.2f} lb/in')

k = 8.00 lb/in


### Calculo de LF

In [1128]:
L_f = Symbol('L_f') # Desconocido
eqt = Eq(k, F_o / (L_f - L_o))  # Ecuación 
print(eqt)
L_f = F_o / k + L_o
print(f'L_f = {L_f :,.2f} in')

Eq(8.0, 12/(L_f - 1.25))
L_f = 2.75 in


### Estimar esfuerzo de diseño

In [1129]:
tau_o = 144e3 # psi

### Asumir diámetro tentativo del resorte

In [1130]:
# Suponer k_b = 1.2 y un c = 7
c = 9.6
k_b = 1.2

d = np.sqrt((8 * k_b * F_o * c)/ (tau_o * np.pi))

print(f'd = {d:.6f} in')
# d = 0.05

d = 0.049443 in


In [1131]:
table_19_2 = pd.read_csv('tables/table_19_2.csv')
table_19_2.head()

,Calibre num,Calibre US alambre in^b,Calibre alambre instrumentos in^b,Calibre Brown & Sharpe in^c,Diámetro métrico mm^d
0,7/0,0.4900,NaN,NaN,13.0
1,6/0,0.4615,0.004,0.5800,12.0
2,5/0,0.4305,0.005,0.5165,11.0
3,4/0,0.3938,0.006,0.4600,10.0
4,3/0,0.3625,0.007,0.4096,9.0


### Buscar un diámetro estándar 

In [1132]:
calibre, *data = table_19_2[table_19_2['Calibre US alambre in^b'] > d] \
    .values[-2]
diameter = data[0]
diameter, calibre

(0.0625, '16')

### Corrección  de tau d

In [1133]:
# tau_o =  8 * k_b * F_o * c / (np.pi * diameter**2)
# print(f'tau_o = {tau_o :,.2f} psi')
tau_o = 21.4 * F_o / diameter**2
print(f'tau_o = {tau_o :,.2f} psi')
# tau_o = 149e3 # psi

tau_o = 65,740.80 psi


### Calcular el numero de esperas máximas activas

In [1134]:
N_a_max = (L_o - 2 * diameter) / diameter
print(f'N_a_max = {N_a_max:.2f}')

N_a = int(N_a_max) - 4#* 0.75
print(f'N_a = {N_a}')

N_a_max = 18.00
N_a = 14


In [1135]:
L_s = diameter * ( N_a )
print(f'L_s = {L_s :,.2f} in')

L_s = 0.88 in


## Valores calculados

### Calcular el valor real de c

In [1136]:
c = pow(
    G * 1e6 * diameter / (8 * k * N_a ), 
    1/3
)
print(f'c = {c :,.2f} in')

c = 9.21 in


### Calcular el valor real de k_b

In [1137]:
k_b = (4 * c + 2) / (4 * c -3)

print(f'k_b = {k_b :,.2f}')

k_b = 1.15


In [1138]:
d_t = np.sqrt((8 * k_b * F_o * c)/ (tau_o * np.pi))

print(f'd = {d_t:.6f} in')

d = 0.070097 in


In [1139]:
D_m = c * diameter
print(f'D_m = {D_m :,.5f} in')

D_e = D_m + diameter
print(f'D_e = {D_e :,.5f} in')

D_i = D_m - diameter
print(f'D_i = {D_i :,.5f} in')

D_m = 0.57563 in
D_e = 0.63813 in
D_i = 0.51313 in


In [1140]:
tau_o = 8 * k_b * F_o * D_m / (np.pi * pow(diameter, 3))
print(f'tau_o = {tau_o :,.2f} psi')

tau_o = 82,693.81 psi


In [1141]:
table_10_4 = pd.read_csv('tables/table_10_4.csv')
table_10_4.head()

,Material,ASTM num,exponente m,diametro in,A kpsi * in^m,Diametro mm,A Mpa * mm^m,Costo relativo,diametro bajo,diametro alto
0,Alambre de piano,A228,0.145,0.004-0.256,201,0.10-6.5,2211,2.6,0.004,0.256
1,Alambre T y R en aceite,A229,0.187,0.020-0.500,147,0.5-12.7,1 855,1.3,0.020,0.500
2,Alambre estirado duro,A227,0.190,0.028-0.500,140,0.7-12.7,1 783,1.0,0.028,0.500
3,Alambre al cromo- vanadio,A232,0.168,0.032-0.437,169,0.8-11.1,2 005,3.1,0.032,0.437
4,Alambre al cromo- silicio,A401,0.108,0.063-0.375,202,1.6-95,1974,40,0.063,0.375


In [1142]:
F_s = k * (L_f - L_s)
print(f'F_s = {F_s :,.2f} lb')

F_s = 15.00 lb


In [1143]:
tau_s = tau_o * (F_s / F_o)
print(f'tau_s = {tau_s :,.2f} psi')

tau_s = 103,367.27 psi


In [1144]:
seleccion = table_10_4[
    (table_10_4['ASTM num'] == 'A313' )
    & (diameter < table_10_4['diametro alto']) 
    & (diameter > table_10_4['diametro bajo'])
].values[0]

m = seleccion[2]
A = seleccion[4]
print(f'm = {m :,.3f} ', f'A = {A :,.2f} kpi * in^m')

S_ut = A / pow(diameter, m)
print(f'S_ut = {S_ut :,.5f} kpsi')

S_sy = porcentaje_sut * S_ut
print(f'S_sy = {S_sy :,.5f} kpsi')

N_y = S_sy / tau_s * 1e3
print(f'N_y = {N_y :,.2f} ')

m = 0.146  A = 169.00 kpi * in^m
S_ut = 253.33093 kpsi
S_sy = 164.66511 kpsi
N_y = 1.59 


In [1145]:
# relación de pandeo
pandeo_relation = 5.26 * D_m
print('L_f', Fore.LIGHTCYAN_EX, L_f, Fore.RESET, '5.26 * D_m', Fore.LIGHTCYAN_EX, pandeo_relation, Fore.RESET, L_f < pandeo_relation)

cc = (L_o - L_s) / N_a
print(f'holgura espira cc = {cc :,.3f} in')
comprobacion = diameter / 10
print(f'comprobacion = {comprobacion :,.3f} in' , cc > comprobacion)

l_o_l_s = L_o - L_s
print(f'l_o - l_s = {l_o_l_s :,.3f} in')
l_f_l_s = 0.15 * (L_f - L_s)
print(f'l_f - l_s = {l_f_l_s :,.3f} in')
print(f'Deflexion total = {l_o_l_s > l_f_l_s}')

L_f  2.75  5.26 * D_m  3.0278133879450673  True
holgura espira cc = 0.027 in
comprobacion = 0.006 in True
l_o - l_s = 0.375 in
l_f - l_s = 0.281 in
Deflexion total = True


In [1146]:
relative_cost = pow(np.pi, 2) * pow(diameter, 2) * N_a * D_m / 4

print(f'Costo relativo = {relative_cost :,.2f} USD')

Costo relativo = 0.08 USD
